### Prompt Engineering

Prompt Engineering is the art and science of creating effective instructions for large language models (LLMs). It's an essential skill for obtaining accurate, relevant, and useful results when interacting with generative AI.

**What is Prompt Engineering?**

Prompt Engineering involves the careful design of instructions (prompts) that guide AI model behavior to produce desired outputs. This includes word choice, information structuring, providing context, and defining output formats.

**Why is it Important?**

- **Accuracy**: Well-crafted prompts produce more accurate results
- **Consistency**: Structured techniques ensure predictable results
- **Efficiency**: Reduces the need for multiple attempts
- **Control**: Enables greater control over style and output format

**Components of a Good Prompt:**

1. **Context**: Relevant background information
2. **Instruction**: What you want the model to do
3. **Examples**: Demonstrations of the desired format (when applicable)
4. **Constraints**: Specific restrictions or guidelines
5. **Output Format**: How the response should be structured

In the following sections, we'll explore various advanced Prompt Engineering techniques:

In [ ]:
# Azure OpenAI Configuration
import json
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv(dotenv_path="../.env")

# Settings
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("MODEL_API_VERSION")
deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")

# Initialize client
client = AzureOpenAI(
    azure_endpoint=azure_endpoint, 
    api_key=api_key,  
    api_version=api_version
)

print("Azure OpenAI client configured successfully!")

### Zero-Shot Prompting

Zero-Shot Prompting is a technique where you provide a task to the language model without prior examples. The model must understand and execute the task based solely on its general understanding and pre-trained knowledge.

**Characteristics:**
- Does not require input/output examples
- Relies on the model's pre-trained knowledge
- Simple to implement
- May not be effective for complex or specific tasks

**Example:**
```
Prompt: "Translate the following sentence to Portuguese: 'Hello, how are you?'"
Response: "Olá, como você está?"
```

**When to use:**
- Simple and well-defined tasks
- When you don't have examples available
- For initial testing of model capabilities

In [ ]:
# Practical example of Zero-Shot Prompting
print("=== TEST: Zero-Shot Prompting ===")

# Example 1: Translation
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": "Translate the following sentence to Portuguese: 'Hello, how are you?'"}
    ],
    temperature=0.3
)

print("Translation:")
print(response.choices[0].message.content)
print("\n" + "="*50 + "\n")

# Example 2: Sentiment classification
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": "Classify the sentiment of this sentence as positive, negative, or neutral: 'This product is amazing, I highly recommend it!'"}
    ],
    temperature=0.1
)

print("Sentiment classification:")
print(response.choices[0].message.content)

### Few-Shot Prompting

Few-Shot Prompting is a technique where you provide a few examples of input and expected output before presenting the actual task. This helps the model better understand the pattern and desired response format.

**Characteristics:**
- Includes 2-5 demonstrative examples
- Improves accuracy compared to zero-shot
- Helps the model understand output format
- Effective for tasks that follow specific patterns

**Example:**
```
Prompt: 
"Classify the sentiment of the following sentences:
Sentence: 'I love this product!' → Sentiment: Positive
Sentence: 'This service is terrible.' → Sentiment: Negative
Sentence: 'The product is ok.' → Sentiment: Neutral
Sentence: 'I am very happy with the purchase!' → Sentiment: ?"
```

**When to use:**
- Tasks that require specific format
- When zero-shot doesn't provide adequate results
- For classification or structured tasks

In [ ]:
# Practical example of Few-Shot Prompting
print("=== TEST: Few-Shot Prompting ===")

# Example: Sentiment classification with examples
few_shot_prompt = """Classify the sentiment of the following sentences:

Sentence: 'I love this product!' → Sentiment: Positive
Sentence: 'This service is terrible.' → Sentiment: Negative  
Sentence: 'The product is ok.' → Sentiment: Neutral
Sentence: 'I couldn't use the app, very confusing.' → Sentiment: Negative
Sentence: 'Worked perfectly, exactly as expected!' → Sentiment: Positive

Sentence: 'The service was reasonable, nothing special.' → Sentiment: ?"""

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": few_shot_prompt}
    ],
    temperature=0.1
)

print("Classification with Few-Shot:")
print(response.choices[0].message.content)
print("\n" + "="*50 + "\n")

# Example 2: Data formatting with examples
format_prompt = """Convert the data to the specified format:

Name: John Silva, Age: 30, City: New York → {"name": "John Silva", "age": 30, "city": "New York"}
Name: Mary Santos, Age: 25, City: Los Angeles → {"name": "Mary Santos", "age": 25, "city": "Los Angeles"}
Name: Peter Costa, Age: 45, City: Chicago → {"name": "Peter Costa", "age": 45, "city": "Chicago"}

Name: Ana Oliveira, Age: 28, City: Miami → ?"""

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": format_prompt}
    ],
    temperature=0.1
)

print("Formatting with Few-Shot:")
print(response.choices[0].message.content)

### Chain-of-Thought Prompting

Chain-of-Thought (CoT) Prompting is a technique that encourages the model to show its step-by-step reasoning before arriving at the final answer. This significantly improves performance on complex reasoning tasks.

**Characteristics:**
- Breaks complex problems into smaller steps
- Shows the reasoning process
- Improves accuracy in mathematical and logical problems
- Allows identification of where reasoning might have failed

**Example:**
```
Prompt: "Solve step by step: If a train travels at 60 km/h and needs to cover 180 km, how long will it take?

Let's think step by step:
1. Speed = 60 km/h
2. Distance = 180 km
3. Time = Distance ÷ Speed
4. Time = 180 ÷ 60 = 3 hours

Answer: 3 hours"
```

**When to use:**
- Mathematical or logical problems
- Tasks requiring multi-step reasoning
- When you need to verify the thought process

In [ ]:
# Practical example of Chain-of-Thought Prompting
print("=== TEST: Chain-of-Thought Prompting ===")

# Example 1: Mathematical problem
cot_prompt = """Solve step by step: 

In a store, there are 24 t-shirts. 1/3 of them are blue, 1/4 are red, and the rest are white. 
If the price of each blue t-shirt is $30, each red one $25, and each white one $20, what is the total stock value?

Let's think step by step:"""

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": cot_prompt}
    ],
    temperature=0.1
)

print("Solution with Chain-of-Thought:")
print(response.choices[0].message.content)
print("\n" + "="*50 + "\n")

# Example 2: Logic problem
logic_prompt = """Solve this logic problem step by step:

Ana, Bruno, and Carlos are in a line. 
- Ana is not in front
- Bruno is not in the middle
- Carlos is not behind

What is the order of the line?

Let's analyze each clue:"""

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": logic_prompt}
    ],
    temperature=0.1
)

print("Logic solution with Chain-of-Thought:")
print(response.choices[0].message.content)

### Meta Prompting

Meta Prompting is an advanced technique where the model is instructed to generate or improve prompts. Essentially, you use the model to create better prompts for itself or other tasks.

**Characteristics:**
- The model helps create more effective prompts
- Can iteratively improve prompt quality
- Useful for automatic prompt optimization
- Requires knowledge about prompting techniques

**Example:**
```
Prompt: "Create an effective prompt to make an AI model explain complex scientific concepts to 10-year-old children. The prompt should include:
- Simple language
- Age-appropriate analogies
- Clear structure
- Interactive elements"

Response: "Explain [scientific concept] to a 10-year-old child. Use simple words, compare with things they know from daily life, organize into 3 main parts, and ask questions to maintain interest."
```

**When to use:**
- Optimization of existing prompts
- Creating prompts for specific tasks
- When you need multiple variations of a prompt

In [ ]:
# Practical example of Meta Prompting
print("=== TEST: Meta Prompting ===")

# Example 1: Creating a prompt for teaching
meta_prompt = """Create an effective prompt to make an AI model explain programming concepts to beginners. The prompt should include:
- Simple and accessible language
- Real-world analogies
- Clear and didactic structure  
- Practical examples
- Comprehension verification

Provide only the optimized prompt, without additional explanations."""

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": meta_prompt}
    ],
    temperature=0.7
)

generated_prompt = response.choices[0].message.content
print("Prompt generated by Meta Prompting:")
print(generated_prompt)
print("\n" + "="*50 + "\n")

# Example 2: Testing the generated prompt
test_content = generated_prompt + "\n\nConcept: Variables in programming"

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": test_content}
    ],
    temperature=0.5
)

print("Test of the generated prompt:")
print(response.choices[0].message.content)

### Prompt Chaining

Prompt Chaining is a technique where you break a complex task into multiple sequential prompts, where the output of one prompt serves as input for the next.

**Characteristics:**
- Divides complex tasks into smaller steps
- The output of one prompt feeds the next
- Allows greater control over each step
- Reduces the chance of errors in complex tasks

**Example:**
```
Prompt 1: "Analyze this text and identify the main points: [text]"
Output 1: "Main points: A, B, C"

Prompt 2: "Based on the main points: A, B, C, create a 100-word executive summary"
Output 2: [Executive summary]

Prompt 3: "Transform this summary into a 3-slide presentation: [summary]"
```

**When to use:**
- Tasks involving multiple processing steps
- When you need granular control over each phase
- For data transformation into multiple formats
- Complex analysis requiring validation at each step

In [ ]:
# Practical example of Prompt Chaining
print("=== TEST: Prompt Chaining ===")

# Sample text for analysis
sample_text = """
Artificial intelligence is rapidly transforming various sectors of the economy. 
In healthcare, machine learning algorithms are being used for more accurate diagnoses 
and discovery of new medicines. In the financial sector, AI helps in fraud detection 
and risk analysis. In education, intelligent systems personalize learning for each 
student. However, ethical challenges and questions about the future of work also arise.
"""

# Prompt 1: Identify main points
print("1. Identifying main points...")
response1 = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": f"Analyze this text and identify the 3 main points in list format:\n\n{sample_text}"}
    ],
    temperature=0.3
)

main_points = response1.choices[0].message.content
print("Main points identified:")
print(main_points)
print("\n" + "-"*30 + "\n")

# Prompt 2: Create executive summary
print("2. Creating executive summary...")
response2 = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": f"Based on the main points identified:\n{main_points}\n\nCreate an executive summary of exactly 50 words."}
    ],
    temperature=0.3
)

executive_summary = response2.choices[0].message.content
print("Executive summary:")
print(executive_summary)
print("\n" + "-"*30 + "\n")

# Prompt 3: Create presentation
print("3. Creating presentation structure...")
response3 = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": f"Transform this summary into a 3-slide presentation structure with titles and bullet points:\n\n{executive_summary}"}
    ],
    temperature=0.5
)

print("Presentation structure:")
print(response3.choices[0].message.content)

### Tree of Thoughts (ToT)

Tree of Thoughts (ToT) is an advanced technique that allows the model to explore multiple reasoning paths simultaneously, like a decision tree, evaluating and choosing the best paths.

**Characteristics:**
- Explores multiple approaches simultaneously
- Allows backtracking when a path doesn't work
- Evaluates the quality of each "thought" or step
- More robust than linear Chain-of-Thought

**Example:**
```
Problem: "How to resolve conflict between teams?"

Thought 1: Direct mediation
├── Evaluation: May be confrontational
├── Next step: Formal meeting
└── Expected result: Quick resolution

Thought 2: Indirect facilitation
├── Evaluation: Less stressful
├── Next step: Individual conversations
└── Expected result: Gradual understanding

Thought 3: Process restructuring
├── Evaluation: Systematic solution
├── Next step: Workflow analysis
└── Expected result: Future prevention

Best path: Combination of 2 and 3
```

**When to use:**
- Complex problems with multiple possible solutions
- When you need to explore alternatives
- Strategic planning and decision making

In [ ]:
# Practical example of Tree of Thoughts (ToT)
print("=== TEST: Tree of Thoughts (ToT) ===")

# Complex problem for analysis
tot_prompt = """Problem: A technology company is facing high employee turnover (30% per year). 
As CEO, you need to solve this quickly.

Explore 3 different approaches simultaneously:

APPROACH 1 - Salary Improvement:
- Evaluate: Pros, cons, and feasibility
- Specific next steps
- Expected outcome

APPROACH 2 - Work Environment Improvement:
- Evaluate: Pros, cons, and feasibility  
- Specific next steps
- Expected outcome

APPROACH 3 - Development Program:
- Evaluate: Pros, cons, and feasibility
- Specific next steps
- Expected outcome

FINAL ANALYSIS:
- Compare the 3 approaches
- Recommend the best strategy or combination
- Justify your choice"""

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": tot_prompt}
    ],
    temperature=0.7,
    max_tokens=1000
)

print("Tree of Thoughts analysis:")
print(response.choices[0].message.content)
print("\n" + "="*50 + "\n")

# Example 2: Planning problem
planning_prompt = """You need to plan the launch of a new mobile app product in 6 months.
Explore simultaneously 3 go-to-market strategies:

STRATEGY A - Gradual Launch:
- Feasibility analysis: 
- Required resources:
- Risks and mitigations:

STRATEGY B - Massive Launch:
- Feasibility analysis:
- Required resources: 
- Risks and mitigations:

STRATEGY C - Niche Launch:
- Feasibility analysis:
- Required resources:
- Risks and mitigations:

DECISION: Choose the best strategy based on the analysis."""

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": planning_prompt}
    ],
    temperature=0.6,
    max_tokens=800
)

print("Planning with Tree of Thoughts:")
print(response.choices[0].message.content)

### Retrieval Augmented Generation (RAG)

Retrieval Augmented Generation (RAG) is a technique that combines text generation from the model with information retrieval from an external knowledge base, enabling more accurate and up-to-date responses.

**Characteristics:**
- Combines generation with information retrieval
- Accesses external and updated knowledge
- Reduces model hallucinations
- Allows citing specific sources

**RAG Components:**
1. **Knowledge Base**: Documents, articles, databases
2. **Retrieval System**: Searches for relevant information
3. **Generator**: Language model that creates the response
4. **Integration**: Combines retrieved information with generation

**Process Example:**
```
Question: "What are the Azure AI updates in 2024?"

1. Retrieval: Search recent documents about Azure AI
2. Context: "Azure AI Foundry was launched in 2024..."
3. Prompt: "Based on the information: [context], answer: [question]"
4. Response: Generated based on retrieved context
```

**When to use:**
- When you need updated information
- To reduce hallucinations
- In corporate Q&A systems
- When the model needs to cite specific sources

In [ ]:
# Practical example of RAG (Simulated)
print("=== TEST: Retrieval Augmented Generation (RAG) ===")

# Simulating a knowledge base (normally would be retrieved from a vector database)
knowledge_base = """
DOCUMENT 1 - Azure AI Foundry (2024):
Azure AI Foundry is a unified platform for developing generative AI applications. 
Launched in 2024, it offers integrated tools for training, evaluating, and deploying AI models.
Includes features like prompt flow, automated evaluation, and model monitoring.

DOCUMENT 2 - Azure OpenAI Service:
Azure OpenAI Service provides access to GPT-4, GPT-3.5-turbo, DALL-E, and Codex models through REST APIs.
Offers enterprise features like virtual networks, customer-managed keys, and compliance.
Available in multiple regions with different models and capabilities.

DOCUMENT 3 - Prompt Engineering Best Practices:
Essential techniques include few-shot learning, chain-of-thought, and prompt chaining.
Important to be specific, use clear examples, and structure instructions well.
The order of information in the prompt can significantly affect results.
"""

# User question
question = "What are the main features of Azure AI Foundry launched in 2024?"

# RAG prompt: Combining retrieved context with the question
rag_prompt = f"""Based on the information provided below, answer the user's question accurately and cite relevant sources.

RETRIEVED CONTEXT:
{knowledge_base}

QUESTION: {question}

INSTRUCTIONS:
- Use only the information provided in the context
- Cite the specific document when relevant
- If information is not available in the context, clearly indicate so
- Be precise and objective in your response"""

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": rag_prompt}
    ],
    temperature=0.2
)

print("RAG Response:")
print(response.choices[0].message.content)
print("\n" + "="*50 + "\n")

# Example 2: Question not in context
question2 = "What is the price of Azure AI Foundry?"

rag_prompt2 = f"""Based on the information provided below, answer the user's question.

RETRIEVED CONTEXT:
{knowledge_base}

QUESTION: {question2}

INSTRUCTIONS:
- Use only the information provided in the context
- If information is not available, respond: "Information not found in the provided context"
"""

response2 = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": rag_prompt2}
    ],
    temperature=0.1
)

print("Test with unavailable information:")
print(response2.choices[0].message.content)

### Active-Prompt

Active-Prompt is a technique that automatically selects the most useful and informative examples to use in few-shot prompting, based on the model's uncertainty about certain questions.

**Characteristics:**
- Automatic selection of most useful examples
- Based on model uncertainty
- Improves few-shot learning efficiency
- Reduces need for manual example curation

**How it Works:**
1. **Uncertainty Analysis**: Identifies where the model has the most doubts
2. **Example Selection**: Chooses examples that address these uncertainties
3. **Adaptive Prompting**: Uses the most relevant examples for each query
4. **Continuous Refinement**: Improves based on feedback

**Conceptual Example:**
```
Task: Sentiment classification

Model identifies uncertainty in:
- Sarcasm
- Ambiguous language
- Cultural expressions

Active-Prompt selects examples that specifically address:
- "What a wonderful day, it rained at my wedding!" (Sarcasm → Negative)
- "Not sure if I liked it..." (Ambiguous → Neutral)
- "It's all good!" (Cultural → Positive)
```

**When to use:**
- When you have many examples available
- To automatically optimize few-shot prompts
- In domains where uncertainty varies by topic
- For adaptive systems that improve with use

In [ ]:
# Practical example of Active-Prompt (Simulated)
print("=== TEST: Active-Prompt ===")

# Simulating the process of selecting examples based on uncertainty
def simulate_active_prompt():
    # Pool of available examples
    example_pool = [
        ("What a wonderful day, it rained at my wedding!", "Negative", "sarcasm"),
        ("This product is ok, nothing special.", "Neutral", "ambiguity"),
        ("I loved the experience!", "Positive", "direct"),
        ("This app is pretty cool!", "Positive", "informal_language"),
        ("Not sure if I really liked it...", "Neutral", "uncertainty"),
        ("Simply fantastic!", "Positive", "enthusiasm"),
        ("Could be better, right?", "Negative", "indirect_criticism")
    ]
    
    # Question that generates uncertainty (informal language + potential sarcasm)
    query = "This thing is pretty awesome, dude!"
    
    # Active-Prompt selects most relevant examples for similar cases
    selected_examples = [
        ("What a wonderful day, it rained at my wedding!", "Negative"),  # sarcasm
        ("This app is pretty cool!", "Positive"),  # informal language
        ("Could be better, right?", "Negative")  # ambiguous tone
    ]
    
    return query, selected_examples

# Running simulation
query, selected_examples = simulate_active_prompt()

print("Example of Active-Prompt in action:")
print(f"Sentence to classify: '{query}'")
print("\nExamples automatically selected based on uncertainty:")
for i, (sentence, sentiment) in enumerate(selected_examples, 1):
    print(f"{i}. '{sentence}' → {sentiment}")

# Building prompt with dynamically selected examples
active_prompt = "Classify the sentiment (Positive, Negative, Neutral) considering cultural context and possible sarcasm:\n\n"

for sentence, sentiment in selected_examples:
    active_prompt += f"Sentence: '{sentence}' → Sentiment: {sentiment}\n"

active_prompt += f"\nSentence: '{query}' → Sentiment: ?"

print(f"\n{'-'*50}")
print("Dynamically constructed prompt:")
print(active_prompt)

# Running classification
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": active_prompt}
    ],
    temperature=0.1
)

print(f"\n{'-'*50}")
print("Active-Prompt result:")
print(response.choices[0].message.content)
print("\n" + "="*50 + "\n")

# Comparison: same prompt without Active-Prompt (generic examples)
generic_prompt = """Classify the sentiment (Positive, Negative, Neutral):

Sentence: 'I love this product!' → Sentiment: Positive
Sentence: 'This service is terrible.' → Sentiment: Negative
Sentence: 'The product is ok.' → Sentiment: Neutral

Sentence: 'This thing is pretty awesome, dude!' → Sentiment: ?"""

response_generic = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": generic_prompt}
    ],
    temperature=0.1
)

print("Comparison with generic Few-Shot:")
print(response_generic.choices[0].message.content)
print("\nNote: Active-Prompt selected more relevant examples to handle informal language and possible sarcasm.")

### Best Practices

**Be Specific.** Leave as little as possible to interpretation. Constrain the operating space.

**Be Descriptive.** Use analogies to make instructions clearer.

**Reinforce Instructions.** Sometimes it may be necessary to repeat yourself for the model. Provide instructions before and after your main content, use an instruction and a cue, etc.

**Order Matters.** The order in which you present information to the model can impact the outcome. Whether you place instructions before your content ("summarize the following...") or after ("summarize the text above...") can make a difference in the result. Even the order of few-shot examples can matter. This is known as recency bias.

**Offer an Alternative to the Model.** Sometimes it can be useful to give the model an alternative path if it cannot complete the assigned task. For example, when asking a question about a text, you might include something like "answer with 'not found' if the answer is not present." This can help the model avoid generating false responses.

In [ ]:
# 🧪 EXPERIMENTATION AREA
# Use this space to test Prompt Engineering techniques

# Example: Test your own technique here
# response = client.chat.completions.create(
#     model=deployment_name,
#     messages=[
#         {"role": "user", "content": "Your prompt here..."}
#     ],
#     temperature=0.5
# )
# 
# print(response.choices[0].message.content)

# Your code here...